# RSA

Accéder à son compte bancaire en ligne, communiquer avec ses proches via une messagerie sécurisée, mettre à jour son dépot git... Derrière toutes ses tâches du quotidien se cachent des protocoles réseaux utilisant des mécanismes cryptographiques. On citera par exemple [TLS](https://fr.wikipedia.org/wiki/Transport_Layer_Security) ou encore [SSH](https://fr.wikipedia.org/wiki/Secure_Shell).

Ces deux exemples utilisent un chiffrement **asymétrique**. Pouvez-vous rappeler ce que c'est?

**Ecrivez votre réponse ici:**
>
>
>

Dans ce premier TP, nous allons tenter de comprendre plus en détail le fonctionnement d'un chiffrement asymétrique, en l'occurence [RSA](https://fr.wikipedia.org/wiki/Chiffrement_RSA), mais également de se convaincre expérimentalement de sa robustesse.

## Le protocole RSA: rappels

Voici les grandes étapes du chiffrement RSA. Cependant, certaines parties sont manquantes! Complétez les trous.

* Bob choisit $p$ et $q$ deux grands nombres ... (plus de 100 chiffres)
* Bob calcule $n=p*q$. 
* Le nombre n, **le modulo RSA**, doit posséder plusieurs centaines de chiffres. Il est public tandis que $p$ et $q$ sont conservés secrets !
* Bob calcule $\varphi(n) = (p-1)(q-1)$, qui doit rester secret. Retrouver $n$ sans connaître $p$ et $q$ est aussi difficile que de factoriser $n$.
* Bob choisit $e$ en s'assurant que $PGDC(e, \phi(n)) = 1$. Il s'agit de **l'exposant d'encryptage RSA**.
* Bob calcule $d$, inverse de $e$ modulo $\phi(n)$ et garde secret cette valeur. Il s'agit de **la clé ... RSA** qui va permettre de décoder par la suite les messages transmis par Alice.
* Bob transmet (ou publie dans un annuaire) le couple $(n,e)$. Ce couple s'appelle **la clé publique RSA.**
* Alice convertit son message "texte" en un nombre $M$ compris entre 0 et $n$.
* Alice calcule $C\equiv M^{e}\mod n$ et envoie ce message crypté $C$.
* Pour le décoder, Bob calcule $M=C^{d} \mod n$ à l'aide de sa clé privée $d$. Ceci lui permet de retrouver le message d'origine car:
$$C^{d} \equiv (M^{e})^d\equiv M^{e.d} \equiv  M \mod n$$
* Bob n'a plus qu'à reconvertir ce nombre en un message clair.

Comme vous le voyez, nous avons besoin de plusieurs outils:


- l'inverse modulaire
  - pour cela il nous faudra implémenter l'algorithme d'Euclide étendu
- l'exponentiation modulaire
- la génération de nombres premiers
  - pour cela, on a besoin de pouvoir tester qu'un nombre est premier

Dans la première partie du tp, nous allons voir comment implémenter ces différents outils.

Dans la deuxième partie, nous essayerons de mieux comprendre les garanties de sécurité de ce protocole ainsi que l'importance du choix des paramètres (notamment les deux nombres premiers).

## Quelques outils

Avant d'implémenter l'algorithme RSA, nous allons devoir utiliser divers outils arithmétiques. Voici une implémentation possible de ces outils.

### L'algorithme d'euclide étendu

L'**algorithme d'Euclide étendu** est une version de l'algorithme d'Euclide; à partir de deux entiers a et b, l'algorithme calcule leur plus grand commun diviseur (P.G.C.D.) ainsi que deux entiers (les coefficients de Bezout) $u$ et $v$ tels que $a*u + b*v = pgcd(a,b)$. L'algorithme d'Euclide permet d'obtenir de tels entiers parce qu'à chaque étape de l'algorithme, on n'a que des sommes de multiples de a et b.


#### Rappels

1. Rappeler la définition de l'inverse modulaire $x^1 \mod n$
2. Quelle est la condition pour que $x$ est un inverse dans $\Z_n$?
3. Comment utiliser les coefficients de Bezout afin de calculer l'inverse modulaire de $x$ dans $\Z_n$?

**Réponses:**




#### Version itérative

1. En vous appuyant de votre cours, compléter fonction `euclide_etendu(a,b)` qui utilise cette méthode itérative. Votre fonction renverra le pgcd de a et de b ainsi que les coefficients de Bezout associés.

In [3]:
# Ecrivez votre code ici

def euclide_etendu(a,b):
    """Renvoie un triplet (d,u,v) avec d=pgcd(a,b) et u et v les coefficients de Bezout"""
    ua, va = 1, 0 
    ub, vb = 0, 1 
    while b != 0:
        aux1, aux2 = ub, vb
        ub, vb = ua - (a//b) * ub, va - (a//b) * vb 
        ua, va = aux1, aux2
        a, b = b, a%b
        
    return a, ua , va

euclide_etendu(120,23)

(1, -9, 47)

### Forgeons nos premiers outils

Nous allons maintenant implémenter différents outils de calcul qui nous seront notamment utile pour la partie sur le protocole RSA.

Nous vous proposons une implémentation en Python des fonctions suivantes:

* `est_premier(n)`: renvoie `True` si `n` est premier, `False` sinon. On rappelle qu'un nombre entier est premier si et seulement s'il n'est divisible que par 1 et lui-même. 
* `pgcd(a,b)`: renvoie un entier correspondant au plus grand diviseur commun de `a` et `b`.
* `mod_inverse(a,b)`: renvoie l'inverse modulaire de `a` et de `b` (indice: pensez au coefficient de Bezout!)

Il n'y a plus qu'à les compléter!

In [4]:
# Ecrivez votre code ici

def est_premier(x):
    return x != 1 and all( x % i != 0 for i in range(2,int(math.sqrt(x))+1))

# pgcd avec euclide_etendu
def pgcd(a,b):
    return euclide_etendue(a,b)[0]

def mod_inverse(a,b):
    """renvoie l'inverse de a modulo b si il existe, None sinon"""
    r,u,v = euclide_etendu(a,b)
    if r == 1:
        return u % b
    else:
        return None


assert 14==mod_inverse(120,23) 

### Un outil plus technique: l'exponentiation modulaire

Vous devez proposer une fonction Python permettant de calculer l'**exponentiation modulaire** étant donné une base $b$, un exposant $e$ et un entier positif $n$ (aussi appelé *modulus*). Ce calcul est notamment utilisé dans le protocole [RSA](https://en.wikipedia.org/wiki/RSA_(cryptosystem)), que nous étudierons plus en détails par la suite.


Commencons par implémenter la méthode la plus simple, `exp_mod_naive(b,e,n)`, qui consiste à effectuer le calcul $b^e$ puis d'appliquer le modulo. 

In [2]:
def exp_mod_naive(b,e,n):
    return (b**e) % n

Nous allons mesurer le temps d'exécution pour des grandes valeurs de b et de e.

In [12]:
import time

def temps_execution(fonction,*parametres):
    t = time.time()
    fonction(*parametres)
    print(time.time()-t)

temps_execution(exp_mod_naive,10000000000000000000,1000000,301)

16.17105007171631


Normalement, votre fonction doit prendre plusieurs secondes pour s'exécuter. Nous allons maintenant implémenter une version encore plus rapide, que nous nommerons `exp_mod_ultra_opt()`. Cette version se base sur la remarque suivante. Soit $b,p,n$ des entiers. Soit $p = \sum_0^{k} 2^i\cdot a_i$ la décomposition de $p$ en puissance de $2$.

$ b^p \mod n = b^{\sum 2^i\cdot a_i} \mod n $

$b^p \mod n = (b^{2^0\cdot a_0}\cdot b^{2^1\cdot a_1} \cdot \ldots \cdot b^{2^{k}\cdot a_{k}}) \mod n $

$b^p \mod n = (b^{2^0\cdot a_0} \mod n) \cdot (b ^ {2^1\cdot a_1}\mod n) \cdot \ldots \cdot (b ^ {2^k\cdot a_k}\mod n) \mod n $ 

1. Quel est le nombre de bits nécessaire pour représenter p? En déduire le nombre de multiplications à effectuer.
2. Proposer une implémentation utilisant cette remarque.

In [ ]:
def exp_mod_ultra_opt(b, p, n):
    res = 1
    while p > 0:
        if p % 2 == 1:
            res = (res*b) % n 
        b = (b*b)%n
        p = p // 2
    return res

9.632110595703125e-05


In [ ]:

temps_execution(exp_mod_ultra_opt,10000000000000000000,1000000,301)

### Génération de clés RSA

1. Ecrire une fonction `genere_cle()` qui renvoie le triplet `(n,e,d)` d'un chiffrement RSA valide. 
1. Quelle condition doit vérifier le message par rapport à $n$ ? Que faire sinon ? (on ne demande pas de réimplémenter toutes les fonctions ici).

In [ ]:
from  random import *

def genere_premier(maxi,mini=2):
    """génère un nombre premier aléatoirement entre mini et maxi"""
    x = 1
    while not est_premier(x):
        x = randint(mini,maxi)
    return x

def genere_grand_premier(k):
    """génère un grand nombre premier de k bits"""
    return genere_premier(2**k-1,2**(k-1))


def genere_cle(k=1024):
    """génère une clé RSA de taille k bits (par défaut k=1024)
    returns:
        n: produit des dex nombres premiers
        e: clé publique
        d: clé privée 
    """
    p = genere_grand_premier(k)
    q = genere_grand_premier(k)
    n = pq
    phi = (p-1)(q-1)
    e = randint(2,phi)
    while pgcd(e,phi)!=1:
        e =randint(2,phi)
    d = mod_inverse(e,phi)
    return n,e,d

genere_premier(800,700)

n,e,d = genere_cle(20)

exp_mod_ultra_opt(exp_mod_ultra_opt(24,e,n),d,n)

### Pourquoi RAS est sûr? 

Parce qu’il est basé sur une dissymétrie fondamentale entre le temps pris pour créer des clés et le temps pris pour les casser... 

#### Voyons cela

1. Créez une fonction `factorisation(n)` qui factorise l’entier $n$ en deux facteurs premiers $p$ et $q$. On se contentera d'une méthode naïve qui consiste à tester tous les couples d'entiers qui sont premiers.

In [ ]:
# Ecrivez votre code ici

def factorisation(N:int) -> (int,int):
    """renvoie deux entiers p et q (s'ils existent) tel que p*q=N et p et q soient des nombres premiers"""
    if N % 2 == 0:
        return 2,N//2
    i = 3
    end = int(mth,sqrt(N)+1)
    while N%i!=0 and i<end:
        i+=2
    if i>=end:
        return None
    else:
        return i, N//i
    
assert factorisation(35) == (5,7)

Nous allons pour la suite effectuer des mesures du temps d'exécution de certaines fonctions. 

1. Mesurez le temps d'exécution de la fonction `generer_grand_premier(k)` pour les valeurs $k=8$ puis $k=19$. On notera `p1` et `p2` les deux nombres ainsi générés.
1. Mesurez le temps d'exécution de la fonction `factorisation(n)` pour les valeurs `n=p1*p2`
2. Testez avec deux grandes valeurs?
3. Testez avec une grande et un plus petite. Que constate-t-on?
4. Quel ratio obtenez-vous?

In [ ]:
import time
start = time.time()
k = 8
p1 = genere_grand_premier(k)
end = time.time()
t1 = end-start
print(f'temps pour pour {k} p1={p1} : t1 :.2}ms')

start = time.time()
k = 19
p2 = genere_grand_premier(k)
end = time.time()
t2  = end-start

print(f'temps pour pour {k} p2={p2} : {t2 :.2}ms')

start = time.time()
n = p1*p2
res1, res2 = factorisation()
end = time.time()
t3 = end-start

print(f'temps pour factorisation {n} res1={res1} )


Ecrivez une fonction qui teste l’évolution de ce ratio en fonction du doublement de la taille de la clé à chaque étape de test.

In [ ]:
# Ecrivez votre code ici

Tracez cette courbe en vous inspirant du code ci-dessous

In [ ]:
import matplotlib.pyplot as plt
size=[3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]
averagetime=[0.0999,1.3,3.9,39,155,689,2021,5187,16213,86038]
plt.plot(size, averagetime)
plt.show()  

<div class="alert alert-block alert-info">
Le plus grand nombre jamais factorisé est RSA 250, c'est à dire que la clé était de taille 250 chiffres. Vous pouvez trouver ici plus de détails sur les records de factorisations: <a href="http://en.wikipedia.org/wiki/RSA_Factoring_Challenge">http://en.wikipedia.org/wiki/RSA_Factoring_Challenge</a>.
</div>

## Quelques erreurs à éviter

### Quand le chiffrement devient inutile...

Nous avons vu en TD que s'il y a trop peu de message en clair possible, alors tout chiffrement devient inutile. Un des exemples données était des capteurs de température. Les messages contenaient uniquement les mesures de température dans un petit interval de valeurs. Il est donc possible de déchiffrer n'importe quel message C en le comparant avec tous les messages possibles préalablement chiffrés avec la clé publique. C'est ce qu'on appelle une **attaque par dictionnaire**.

1. pouvez-vous proposer une contre-mesure, c'est-à-dire une solution permettant de contrer ce type d'attaque? (indice: il faut brouiller les pistes)



### Voir les choses en grand!

Imaginons qu'un message M ait été chiffré avec RSA avec la clé publique `(n=95,e=23)`. Plus précisément, chaque lettre a été chiffré séparement avec cette clé. Voici le message crypté ainsi obtenu:

[8,29,52,1,74,0,20,37,0,40,12,52]

1. Seriez-vous capable de retrouver le message $M$ initial? (indice: $n$ n'est pas si grand que ça...)
2. Pourquoi ne faut-il jamais utiliser RSA comme cela?

<div class="alert alert-block alert-info">
Même en choisissant de grands nombres premiers, cela ne suffit pas à garantir une sécurité théorique absolue.

En effet, si l'on dispose d'un ordinateur quantique suffisamment puissant, alors il est théoriquement possible de factoriser n'importe quel nombre premier en temps polynomial. C'est le <a href="https://en.wikipedia.org/wiki/Shor%27s_algorithm">théorème de Shore</a>.
</div>

## Pour les plus rapides

### Attaque de RSA pour e petit

Pourquoi RSA peut-il être fragile pour $e$ "petit" ? Par exemple inférieur à 10 ou de l'ordre de quelques dizaines ? Faites des recherches et coder des éléments d'une attaque possible dans ce cas 

### Tests de primalité
On voit que tester la primalité est primordial pour les performances de l'algorithme RSA. 
Reprenons un peu cette étude et testons le temps d'exécution de notre fonction `est_premier(N)` pour une grande valeur de N. 

In [ ]:

est_premier(1051364518442342302270880192779903262713)

Pour résoudre ce problème, implantez le test de primalité probabiliste de Fermat. Le principe de ce test est de choisir aléatoirement et uniformément un entier $a$ et de tester si $a^{n-1} \not\equiv 1 \mod n$. Si c'est le cas, alors $n$ n'est pas premier. On répète ce test $k$ fois. Si aucune itération ne permet de prouver que $n$ est composite, alors il est considéré comme *probablement premier*. Il suffit ensuite de vérifier par une méthode déterministe s'il est vraiment premier ou non.

In [ ]:
def est_premier_fermat(n,k=10):
    pass

On peut ensuite comparer les performances des deux implémentations.

In [ ]:

temps_execution(est_premier,275478901)
temps_execution(est_premier_fermat,275478901)

### A faire chez vous: Utilisation de Openssl

Le logiciel [OpenSSL](https://www.openssl.org/) regroupe des implémentations de méthodes cryptographiques. 

Nous découvrirons cet outil en détails lors du prochain TP. Afin de gagner du temps, cherchez comment effectuer toutes ces tâches:

* Création d’un mot de passe et son hashage
* Création d’une clé rsa
* Utilisation de la clé
* Creer un certificat autosigné et le vérifier
* Récupérer via les parametre de firefox le certificat d’amazone et le verifier:
* Vérifiez le hachage de la clé publique pour vérifier qu’il correspond à un CSR ou à une clé privée
* Créer un CSR pour une demande de signature de clé privée et de certificat